# Языковые модели на n-граммах

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://www.nltk.org/api/nltk.util.html
* https://web.stanford.edu/~jurafsky/slp3/3.pdf
* https://www.youtube.com/watch?v=QGT6XTeA3YQ

## Задачи для совместного разбора

1\. Выделите из текста n-граммы.

In [ ]:
text = """Вода это жидкость которая имеет свойство быть водой.
Вода состоит из молекул, которые выглядят как вода"""

In [ ]:
from nltk.util import ngrams
from nltk import sent_tokenize
from nltk.tokenize import RegexpTokenizer

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# unigrams = ngrams(text, n=1)
# unigrams = list(unigrams)
# unigrams

In [ ]:
sents = sent_tokenize(text)
tokenizer = RegexpTokenizer("\w+")

unigrams = []
bigrams = []
for sent in sents:
  tokens = tokenizer.tokenize(sent.lower())
  unigrams.extend(ngrams(tokens, n=1))
  bigrams.extend(ngrams(tokens, n=2))

In [ ]:
unigrams[:5]

[('вода',), ('это',), ('жидкость',), ('которая',), ('имеет',)]

In [ ]:
bigrams[:5]

[('вода', 'это'),
 ('это', 'жидкость'),
 ('жидкость', 'которая'),
 ('которая', 'имеет'),
 ('имеет', 'свойство')]

2. Рассчитайте вероятности  $P(вода)$, $P(это|вода)$, $P(состоит|вода)$.

$$P(w_i) = \frac{C(w_i)}{N}$$
$$P(w_i|w_{i-1})=\frac{C(w_{i-1} w_i)}{C(w_{i-1})}$$

In [ ]:
w = ("вода", )
c = unigrams.count(w)
N = len(unigrams)
c, N, c / N

(3, 16, 0.1875)

In [ ]:
# P(это|вода)
w2 = ("вода", "это")
c2 = bigrams.count(w2)
c2, c, c2/c

# P(это)

(1, 3, 0.3333333333333333)

3. Рассчитайте вероятности  $P_L(вода)$, $P_L(это|вода)$, $P_L(состоит|вода)$.

$$P_L(w_i) = \frac{C(w_i)+1}{N+V}$$, $$P_L(w_i|w_{i-1})=\frac{C(w_{i-1} w_i)+1}{C(w_{i-1})+V}$$

In [ ]:
len(set(unigrams))

14

In [ ]:
# вода не
# P(не|вода) = 0
# P  = (0 + 1) / (3 + 14)

## Задачи для самостоятельного решения

In [1]:
from nltk.util import ngrams
from nltk import sent_tokenize
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import numpy as np

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

<p class="task" id="1"></p>

1\. Считайте файл `data/moya-semia/Лучше кошки зверя нет 2.csv`. Получите список предложений из сообщений. Приведите предложения к нижнему регистру и удалите все символы, кроме букв и пробелов. Получите список слов (униграмм) и биграмм.

In [3]:
text = pd.read_csv('Лучше кошки зверя нет 2.csv', header=None, usecols=[1])[1].sum()
sents = sent_tokenize(text)
sents[:5]

['Завтра мои котейки идут к врачу.',
 'А 21-го жду их домой.',
 'Прошу помощи.',
 'Чем кормить лучше (корм).',
 'Какой наполнитель использовать.']

In [9]:
tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(sents[1].lower())
tokens_pure = []
for token in tokens:
  try:
    int(token)
  except ValueError:
    tokens_pure.append(token)

tokens_pure, sents[1]

(['а', 'го', 'жду', 'их', 'домой'], 'А 21-го жду их домой.')

In [10]:
tokenizer = RegexpTokenizer(r'\w+')
unigrams = []
bigrams = []
for sent in sents:
  tokens = tokenizer.tokenize(sent.lower())
  tokens_pure = []
  for token in tokens:
    try:
      int(token)
    except ValueError:
      tokens_pure.append(token)
  unigrams.extend(ngrams(tokens_pure, n=1))
  bigrams.extend(ngrams(tokens_pure, n=2))

In [11]:
unigrams[:10]

[('завтра',),
 ('мои',),
 ('котейки',),
 ('идут',),
 ('к',),
 ('врачу',),
 ('а',),
 ('го',),
 ('жду',),
 ('их',)]

In [12]:
bigrams[:5]

[('завтра', 'мои'),
 ('мои', 'котейки'),
 ('котейки', 'идут'),
 ('идут', 'к'),
 ('к', 'врачу')]

<p class="task" id="2"></p>

2\. Получите распределение вероятностей для униграм $P(w_i) = \frac{C(w_i)}{N}$, где $N$ - количество униграм, $C(w_i)$ - частота использования токена $w_i$. Получите распределение условных вероятностей для биграмм $P(w_i|w_{i-1})=\frac{C(w_{i-1} w_i)}{C(w_{i-1})}$ ($C(w_{i-1} w_i)$ - частота использования словосочетания $w_{i-1}w_i$).

In [13]:
from collections import Counter

In [14]:
p_unigrams = Counter(unigrams)
N = len(unigrams)
for k, v in p_unigrams.items():
  p_unigrams[k] = v / N

In [15]:
p_bigrams = Counter(bigrams)
cnt_unigrams = Counter(unigrams)
for k, v in p_bigrams.items():
    p_bigrams[k] = v / cnt_unigrams[(k[0], )]

<p class="task" id="3"></p>

3\.Воспользовавшись полученными вероятностями, сгенерируйте текст длиной не более 20 символов, начинающийся с токена "мой". При генерации текста выбирайте слово с наибольшей вероятностью соответствующего биграмма. Выведите полученный текст на экран.

In [16]:
def choose_word(word, probs):
  res = []
  for k, v in probs.items():
    if k[0] == word:
      res.append((k[1], v))
  return max(res, key=lambda x: x[1])[0]

In [17]:
res = ['мой']
while len(res) < 20:
  res.append(choose_word(res[-1], p_bigrams))
' '.join(res)

'мой кот в доме не знаю что то что то что то что то что то что то что то'

<p class="task" id="4"></p>

4\.Воспользовавшись полученными вероятностями, сгенерируйте текст длиной не более 20 символов, начинающийся с токена "мой". При генерации текста выбирайте слово пропорционально вероятностям соответствующих биграммов. Выведите полученный текст на экран.

In [18]:
def choose_word_prob(word, probs):
  bgms = list(filter(lambda x: x[0] == word, probs))
  bgms_probs = np.array(list(map(lambda x: probs[x], bgms)))
  bgms = list(map(lambda x: x[1], bgms))

  bgms_probs_norm = bgms_probs * (1 / bgms_probs.sum()) # чтобы вероятности в сумме давали 1
  res = np.random.choice(bgms, p=bgms_probs_norm)
  return res

In [21]:
res = ['мой']
while len(res) < 20:
  res.append(choose_word_prob(res[-1], p_bigrams))
' '.join(res)

'мой добытчик все в ней теперь для движения какой успокаивающий эффект на прогулку на место освободил куда шел очень тактичный'

<p class="task" id="5"></p>

5\. Получите распределение вероятностей для униграм, воспользовавшись сглаживанием Лапласа: $P_L(w_i) = \frac{C(w_i)+1}{N+V}$, где $V$ - количество уникальных униграмм. Получите распределение условных вероятностей для биграмм $P_L(w_i|w_{i-1})=\frac{C(w_{i-1} w_i)+1}{C(w_{i-1})+V}$

In [22]:
p_unig_laplace = Counter(unigrams)
N = len(unigrams)
V = len(set(unigrams))
for k, v in p_unig_laplace.items():
  p_unig_laplace[k] = (v + 1) / (N + V)

In [23]:
p_bi_laplace = Counter(bigrams)
cnt_unigrams = Counter(unigrams)
V = len(set(bigrams))
for k, v in p_bi_laplace.items():
    p_bi_laplace[k] = (v + 1) / (cnt_unigrams[(k[0], )] + V)

<p class="task" id="6"></p>

6\.Воспользовавшись полученными после сглаживания вероятностями, сгенерируйте текст длиной не более 20 символов, начинающийся с токена "мой". При генерации текста выбирайте слово пропорционально вероятностям соответствующих биграммов. Выведите полученный текст на экран.

In [25]:
res = ['мой']
while len(res) < 20:
  res.append(choose_word_prob(res[-1], p_bi_laplace))
' '.join(res)

'мой умка такой мой гроб кошек сейчас ведро наполняется вода течет этот болезный а на даче не помогли наконец то'

<p class="task" id="7"></p>

7\. Рассчитайте перплексию для текста "Котя пришел домой с хромой лапой" для четырех моделей: на 1/2-граммах и с/без использования сглаживания Лапласа. Сведите результат в таблицу. Повторите вычисления для текста "После пар я поеду кормить своего кота", используя доступные модели.

$Perplexity(W) = P(w_1w_2...w_N)^{-\frac{1}{N}}$

Для модели на униграммах $P(w_1w_2...w_N) = \Pi_{i=1}^{N}{P(w_i)}$

Для модели на биграммах $P(w_1w_2...w_N) = \Pi_{i=1}^{N}{P(w_i|w_{i-1})}$

In [26]:
test1 = "После пар я поеду кормить своего кота".lower()
test2 = "Котя пришел домой с хромой лапой".lower()

In [27]:
perplexity = []
for test in [test1, test2]:
  perp = {
      'perp_uni': 1,
      'perp_uni_laplace': 1,
      'pepr_bi': 1,
      'perp_bi_laplace': 1
  }
  wi_1 = test.split()[0]
  N = len(test.split())
  V = len(set(bigrams))
  for word in test.split()[1:]:
    perp['perp_uni'] *= p_unigrams[(word, )]
    perp['perp_uni_laplace'] *= p_unig_laplace[(word, )]
    prob = p_bigrams[(wi_1, word, )]
    prob_laplace = p_bi_laplace[(wi_1, word, )]
    if prob == 0:
      perp['pepr_bi'] *= np.inf
    else:
      perp['pepr_bi'] *= prob
    if prob_laplace == 0:
      perp['perp_bi_laplace'] *= 1 / (cnt_unigrams[(wi_1, )] + V)
    else:
      perp['perp_bi_laplace'] *= prob_laplace
    wi_1 = word

  perplexity.append({k: v**(-1/N) for k, v in perp.items()})

perplexity

[{'perp_uni': 1483.630434721017,
  'perp_uni_laplace': 1376.5231653603319,
  'pepr_bi': 0.0,
  'perp_bi_laplace': 16636.92132888866},
 {'perp_uni': 450.7240472993275,
  'perp_uni_laplace': 493.7256987852116,
  'pepr_bi': 15.764256798862787,
  'perp_bi_laplace': 3433.3592302669244}]

In [28]:
pd.DataFrame({'test1': perplexity[0],
              'test2': perplexity[1]})

,test1,test2
perp_uni,1483.630435,450.724047
perp_uni_laplace,1376.523165,493.725699
pepr_bi,0.000000,15.764257
perp_bi_laplace,16636.921329,3433.359230
